In [87]:
import torch
from torch_geometric.utils import scatter

import pathpyG as pp
pp.config['torch']['device'] = 'cpu'

TODOs:
- turn into class functions: (the ones needed to estimate order)
- add as utils (computing of weigthed outdegrees and transition probabobilities)
- unit tests


In [117]:
# dag_data = pp.DAGData(pp.IndexMap(list("abcde")))
dag_data = pp.DAGData(pp.IndexMap(list("01234")))

# walk_1  =('a','b','c','d','e','c','b','a','c','d','e','c','e','d','c','a')
# walk_2  =('a','b','c','d','e','c')
# dag_data.append_walk(walk_1)
# dag_data.append_walk(walk_2)


dag_data.append_walk(list("0230230230230"), weight=2)
dag_data.append_walk(list("1241241241241"), weight=2)
# dag_data.append_walk(list("0430241"), weight=1)

m = pp.MultiOrderModel.from_DAGs(dag_data, max_order=6)


In [118]:
from torch_geometric.loader import DataLoader
dag_graph = next(iter(DataLoader(dag_data.dags, batch_size=len(dag_data.dags)))).to(pp.config["torch"]["device"])

In [125]:
import torch

def compute_weighted_outdegrees(graph):
    """
    Compute the weighted outdegrees of each node in the graph.

    Args:
        graph (Graph): pathpy graph object.

    Returns:
        tensor: Weighted outdegrees of nodes.
    """
    weighted_outdegree = scatter(graph.data.edge_weight, graph.data.edge_index[0], dim=0, dim_size=graph.data.num_nodes, reduce='sum')
    return weighted_outdegree

def compute_transition_probabilities(graph):
    """
    Compute transition probabilities based on weighted outdegrees.

    Args:
        graph (Graph): pathpy graph object.

    Returns:
        tensor: Transition probabilities.
    """
    weighted_outdegree = compute_weighted_outdegrees(graph)
    source_ids = graph.data.edge_index[0]
    return graph.data.edge_weight / weighted_outdegree[source_ids]

def get_zeroth_order_log_likelihood(dag_graph): 
    """
    Compute the zeroth order log likelihood.

    Args:
        dag_graph (DataBatch): Input DAG graph data.

    Returns:
        float: Zeroth order log likelihood.
    """
    # Get frequencies
    # TODO: put this tensor directly in dag_graph (intead of edge_weight) and remove the following line
    frequencies = dag_graph.edge_weight[dag_graph.ptr[:-1]]
    
    # Get ixs starting nodes
    mask = torch.ones(dag_graph.num_nodes, dtype=bool) 
    mask[dag_graph.edge_index[1]] = False
    start_ixs = dag_graph.node_sequence.squeeze()[mask]

    # Compute node emission probabilities
    # TODOL modify once we have zeroth order in mon
    _, counts = torch.unique(dag_graph.node_sequence, return_counts=True)
    node_emission_probabilities = counts / counts.sum()
    return torch.mul(frequencies, torch.log(node_emission_probabilities[start_ixs])).sum().item()

def get_intermediate_order_log_likelihood(m, dag_graph, order):
    """
    Compute the intermediate order log likelihood.

    Args:
        m (MultiOrderModel): Multi-order model.
        dag_graph (DataBatch): Input DAG graph data.
        order (int): Order of the intermediate log likelihood.

    Returns:
        float: Intermediate order log likelihood.
    """
    # Get frequencies
    # TODO: put this tensor directly in dag_graph (intead of edge_weight) and remove the following line
    frequencies = dag_graph.edge_weight[dag_graph.ptr[:-1]]
    
    # Get intermediate HO nodes ixs (indentify ixs of nodes without indegree)
    mask = torch.ones(dag_graph.num_nodes, dtype=bool) 
    mask[dag_graph.edge_index[1]] = False
    ixs = torch.where(mask)[0]
    num_ixs = ixs.shape[0]
    ho_intermediate_ixs = ixs + torch.arange(num_ixs) * order
    print(dag_graph.edge_index.shape, m.layers[order+1].data.inverse_idx.shape)
    m.layers[order+1].data.inverse_idx[ho_intermediate_ixs]
    log_lh_unique_sub_walks =  torch.log(compute_transition_probabilities(m.layers[order])[m.layers[order+1].data.inverse_idx[ho_intermediate_ixs]])
    return torch.mul(frequencies, log_lh_unique_sub_walks).sum().item()

def get_mon_log_likelihood(m, dag_graph, max_order=1):
    """
    Compute the likelihood of the walks given a multi-order model.

    Args:
        m (MultiOrderModel): The multi-order model.
        dag_graph (DataBatch): Dataset containing the walks.
        max_order (int, optional): The maximum order up to which model layers
            shall be taken into account. Defaults to 1.

    Returns:
        float: The log likelihood of the walks given the multi-order model.
    """
    llh = 0

    # Adding likelihood of zeroth order
    llh += get_zeroth_order_log_likelihood(dag_graph)

    # Adding the likelihood for all the intermediate orders
    for order in range(1, max_order):
        llh += get_intermediate_order_log_likelihood(m, dag_graph, order)

    # Adding the likelihood of highest/stationary order
    if max_order > 0:
        llh += (torch.log(compute_transition_probabilities(m.layers[max_order])) * m.layers[max_order].data.edge_weight).sum().item()
    else:
        # Compute likelihood for zeroth order (to be modified)
        # TODO: modify once we have zeroth order in mon 
        # (then won t need to compute emission probs from dag_graph -- which also hinders us from computing the lh that a new set of path swas generated by the model)
        frequencies = dag_graph.edge_weight[dag_graph.ptr[:-1]]
        counts = torch.bincount(dag_graph.node_sequence.T[0], frequencies.repeat_interleave(dag_graph.ptr[1:] - dag_graph.ptr[:-1]))
        node_emission_probabilities = counts / counts.sum()
        llh = torch.mul(torch.log(node_emission_probabilities), counts).sum().item()

    return llh


In [126]:
order = 0
mask = torch.ones(dag_graph.num_nodes, dtype=bool) 
mask[dag_graph.edge_index[1]] = False
ixs = torch.where(mask)[0]
num_ixs = ixs.shape[0]
ho_intermediate_ixs = ixs + torch.arange(num_ixs) * (order - 1) 
ho_intermediate_ixs

tensor([ 0, 12])

In [127]:
for o in range(max(m.layers)+1):  
    print(o,get_mon_log_likelihood(m,dag_graph, max_order=o))

0 -81.78048728363943
1 -17.68498945236206
torch.Size([2, 24]) torch.Size([24])
2 -6.594634532928467
torch.Size([2, 24]) torch.Size([24])
torch.Size([2, 24]) torch.Size([22])
3 -6.594634532928467
torch.Size([2, 24]) torch.Size([24])
torch.Size([2, 24]) torch.Size([22])
torch.Size([2, 24]) torch.Size([20])
4 -6.594634532928467
torch.Size([2, 24]) torch.Size([24])
torch.Size([2, 24]) torch.Size([22])
torch.Size([2, 24]) torch.Size([20])
torch.Size([2, 24]) torch.Size([18])
5 -6.594634532928467
torch.Size([2, 24]) torch.Size([24])
torch.Size([2, 24]) torch.Size([22])
torch.Size([2, 24]) torch.Size([20])
torch.Size([2, 24]) torch.Size([18])
torch.Size([2, 24]) torch.Size([16])


IndexError: index 18 is out of bounds for dimension 0 with size 16

Probably there is somethign wrong in _bipartite_encode

In [10]:
def get_mon_dof(m, max_order=None, assumption="paths"):
    """
    The degrees of freedom fo the kth layer of a multi-order model this depende on the number of different paths of exactly length k in the graph.
    Therefore, we can obtain this values by summing the entries of the kth power of the binary adhacency matrix of the graph.
    Finally, we must consider that, due the conservation of probablility, all non-zero rows of the transition matrix of the higher-order network must sum to one. 
    This poses on additional constraint per row that respects the condition, which should be removed from the total count of degrees of freedom.

    Args:
        m (MultiOrderModel): The multi-order model.
        max_order (int, optional): The maximum order up to which model layers 
            shall be taken into account. Defaults to None, meaning it considers 
            all available layers.
        assumption (str, optional): If set to 'paths', only paths in the 
            first-order network topology will be considered for the degree of 
            freedom calculation. If set to 'ngrams', all possible n-grams will 
            be considered, independent of whether they are valid paths in the 
            first-order network or not. Defaults to 'paths'.

    Returns:
        int: The degrees of freedom for the multi-order model.

    Raises:
        AssertionError: If max_order is larger than the maximum order of 
            the multi-order network.
        ValueError: If the assumption is not 'paths' or 'ngrams'.
    """
    if max_order is None:
        max_order = max(m.layers)
    
    assert max_order <= max(m.layers), "Error: max_order cannot be larger than maximum order of multi-order network"

    dof = m.layers[1].data.num_nodes - 1  # Degrees of freedom for zeroth order

    if assumption == "paths":
        # COMPUTING CONTRIBUTION FROM NUM PATHS AND NONERO OUTDEGREES SEPARATELY
        # TODO: CAN IT BE DONE TOGETHER?

        # Adding dof from Number of paths of length k 
        for k in range(1, max_order + 1):
            
            if k == 1:
                edge_index = m.layers[1].data.edge_index
            else:
                edge_index = m.lift_order_edge_index(edge_index, num_len_k_paths)
            num_len_k_paths = edge_index.shape[1]  # Number of paths of length k
            dof += num_len_k_paths 
        
        # removing dof from total probability of nonzero degree nodes
        for k in range(1, max_order+1):
            
            if k == 1:
                edge_index_adj = m.layers[1].data.edge_index
                edge_index = edge_index_adj
            else:
                edge_index, _ = edge_index @ edge_index_adj
            num_nonzero_outdegrees = torch.unique(edge_index[0]).size(0)
            dof -=  num_nonzero_outdegrees

      
    elif assumption == "ngrams":
        for order in range(1, max_order + 1):
            dof += (m.layers[1].data.num_nodes ** order) * (m.layers[1].data.num_nodes - 1)
    else:
        raise ValueError(f"Unknown assumption {assumption} in input. The only accepted values are 'path' and 'ngram'")

    return int(dof)

for o in range(0,max(m.layers)+1):
    # get_mon_dof(m,max_order=o)
    print("dof",o,"-",get_mon_dof(m,max_order=o))

dof 0 - 4
dof 1 - 5
dof 2 - 8
dof 3 - 13
dof 4 - 20
dof 5 - 31
dof 6 - 46


/opt/conda/lib/python3.10/site-packages/torch_geometric/edge_index.py:784: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(


In [11]:
from scipy.stats import chi2
def lh_ratio_test(mon, dag_graph, max_order_null = 0, max_order = 1, assumption='paths', significance_threshold=0.01):
    assert max_order_null < max_order, 'Error: order of null hypothesis must be smaller than order of alternative hypothesis'
    assert max_order < max(mon.layers), f'Error: order of hypotheses ({max_order_null} and {max_order}) must be smaller than the maximum order of the MultiOrderModel {max(mon.layers)}'
    # let L0 be the likelihood for the null model and L1 be the likelihood for the alternative model

    # we first compute a test statistic x = -2 * log (L0/L1) = -2 * (log L0 - log L1)
    x = -2 * (get_mon_log_likelihood(mon, dag_graph, max_order=max_order_null) - get_mon_log_likelihood(mon, dag_graph, max_order=max_order))

    # we calculate the additional degrees of freedom in the alternative model
    dof_diff = get_mon_dof(m,max_order, assumption = assumption) - get_mon_dof(m,max_order_null, assumption = assumption)
    print(x, dof_diff)

    # if the p-value is *below* the significance threshold, we reject the null hypothesis
    p = 1-chi2.cdf(x, dof_diff)
    return (p<significance_threshold), p

Mess originates from the fact that mon and walk dataset can have incompatbile values. 

In [12]:
lh_ratio_test(m, dag_graph, max_order_null = 0, max_order = 1, assumption='paths', significance_threshold=0.01)

128.19099566255474 1


(True, 0.0)

In [13]:
def estimate_order(mon, walk_data, max_order=None, significance_threshold=0.01):
    """
    Selects the optimal maximum order of a multi-order network model for the
    observed paths, based on a likelihood ratio test with p-value threshold of p
    By default, all orders up to the maximum order of the multi-order model will be tested.

    @param paths: The path statistics for which to perform the order selection

    @param maxOrder: The maximum order up to which the multi-order model shall be tested.
    """
    if max_order == None:
        max_order = mon.max_order
    assert max_order <= max(mon.layers), 'Error: maxOrder cannot be larger than maximum order of multi-order network'
    assert max_order > 1, 'Error: maxOrder must be larger than one'

    max_accepted_order = 1

    # Test for highest order that passes
    # likelihood ratio test against null model
    for k in range(2, max_order+1):
        if lh_ratio_test(m, walk_data, max_order_null = k-1, max_order = k, significance_threshold=significance_threshold)[0]:
            max_accepted_order = k

    return max_accepted_order

estimate_order(m, dag_graph, max_order=max(m.layers)-1) # this -1 indicates that something is still off

22.180709838867188 3
-0.0 5
-0.0 7
-0.0 11


2